In [ ]:
import scipy.io
img_features = scipy.io.loadmat('Features.mat')['out']
bb_features = scipy.io.loadmat('baseline_bb_vgg.mat')['out']
spatial_features = scipy.io.loadmat('5Dvec.mat')['out']

In [ ]:
import numpy as np
image_feat = np.split(img_features[0],16200)
image_feat = np.asarray(image_feat)
print image_feat.shape

In [ ]:
bb_feat = np.split(bb_features[0],16200)
bb_feat = np.asarray(bb_feat)
print bb_feat.shape

In [ ]:
spatial_feat = spatial_features[:16200]
print spatial_feat.shape

In [7]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, RepeatVector
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers import Merge
from keras.optimizers import SGD
from keras.layers.core import Activation
import numpy as np
import h5py

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)


In [8]:
image_model = Sequential()
image_model.add(Dense(1000, input_dim=4096, activation="linear"))

bb_model = Sequential()
bb_model.add(Dense(1000, input_dim=4096, activation="linear"))

temp_image_model = Sequential()
temp_image_model.add(Merge([image_model, bb_model], mode='concat', concat_axis=-1))

spatial_model = Sequential()
spatial_model.add(Dense(5, input_dim=5, activation="linear"))

net_image_model = Sequential()
net_image_model.add(Merge([temp_image_model, spatial_model], mode='concat', concat_axis=-1))
net_image_model.add(RepeatVector(40))
print "Image model made."

Image model made.


In [9]:
vocab_size = 10000
max_caption_len = 40

print "Preparing Language Model."
language_model = Sequential()
language_model.add(Embedding(vocab_size,512, input_length=max_caption_len))
language_model.add(LSTM(output_dim=512, return_sequences=True))
language_model.add(TimeDistributed(Dense(512)))
# language_model.output
print "Language Model set."

Preparing Language Model.
Language Model set.


In [11]:
print "Merging Language and Image Model"
full_model = Sequential()
full_model.add(Merge([net_image_model, language_model], mode='concat', concat_axis=-1))
full_model.add(LSTM(512, return_sequences=False))
full_model.add(Dense(10000))
full_model.add(Activation('softmax'))
print "Models merged..."

Merging Language and Image Model
Models merged...


In [12]:
print "loading language features"
partial_cap = scipy.io.loadmat('partial_cap.mat')['out']
next_words = scipy.io.loadmat('next_words.mat')['out']

print "Language features loaded.."

loading language features
Language features loaded..


In [8]:
partial_cap = partial_cap[:16200]
next_words = next_words[:16200]
print partial_cap.shape, next_words.shape

(16200, 40) (16200, 10000)


In [9]:
full_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
full_model.fit([feat, partial_cap], next_words, batch_size=16, nb_epoch=100)

Epoch 1/20
  256/16200 [..............................] - ETA: 1805s - loss: 63.6818

In [11]:
full_model.save('baseline_weights_final.h')